In [2]:
from datasets import load_dataset
import pandas as pd
# Load 2000 training and 500 test examples for speed
imdb = load_dataset("imdb")
train = pd.DataFrame(imdb["train"].select(range(2000)))
test = pd.DataFrame(imdb["test"].select(range(500)))
print("Train size:", len(train), "Test size:", len(test))
train.head()


C:\Users\helam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\helam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\helam\.cache\huggingface\hub\datasets--imdb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cac

Train size: 2000 Test size: 500


,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [3]:
import re
def clean_text(text):
    text = re.sub(r"<br\s*/?>", " ", text)
    text = re.sub(r"[^\w\s']", "", text)
    return text.lower()
train["text"] = train["text"].apply(clean_text)
test["text"] = test["text"].apply(clean_text)


In [5]:
from snorkel.labeling import labeling_function, LFAnalysis
ABSTAIN, NEG, POS = -1, 0, 1
positive_words = {"great","excellent","amazing","wonderful","best","fantastic"}
negative_words = {"bad","terrible","awful","worst","boring","poor"}
@labeling_function()
def lf_positive(x):
    return POS if any(word in x.text.split() for word in positive_words) else ABSTAIN
@labeling_function()
def lf_negative(x):
    return NEG if any(word in x.text.split() for word in negative_words) else ABSTAIN
@labeling_function()
def lf_exclaim(x):
    return POS if x.text.count("!") > 2 else ABSTAIN

lfs = [lf_positive, lf_negative, lf_exclaim]

In [6]:
from snorkel.labeling import PandasLFApplier
applier = PandasLFApplier(lfs)
L_train = applier.apply(train)
LFAnalysis(L_train, lfs).lf_summary()

100%|██████████| 2000/2000 [00:00<00:00, 6473.12it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
lf_positive,0,[1],0.3210,0.1795,0.1795
lf_negative,1,[0],0.5635,0.1795,0.1795
lf_exclaim,2,[],0.0000,0.0000,0.0000


In [8]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=42)
# Get probabilistic labels
train_probs = label_model.predict_proba(L_train)

INFO:root:Computing O...
INFO:root:Estimating \mu...
100%|██████████| 500/500 [00:00<00:00, 987.54epoch/s] 
INFO:root:Finished Training


In [9]:
train_preds = label_model.predict(L_train)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
# Vectorize
vectorizer = TfidfVectorizer(max_features=5_000)
X_train = vectorizer.fit_transform(train["text"])
y_train = train_preds
# Fit classifier
clf = LogisticRegression(max_iter=200)
clf.fit(X_train, y_train)
# Evaluate on test set
X_test = vectorizer.transform(test["text"])
y_test = test["label"]
preds = clf.predict(X_test)
print(classification_report(y_test, preds, target_names=["neg","pos",'exclaim']))

              precision    recall  f1-score   support

         neg       0.00      0.00      0.00         0
         pos       1.00      0.81      0.90       500
     exclaim       0.00      0.00      0.00         0

    accuracy                           0.81       500
   macro avg       0.33      0.27      0.30       500
weighted avg       1.00      0.81      0.90       500



C:\Users\helam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\helam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\helam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:17

In [19]:
clf_fs = LogisticRegression(max_iter=200)
clf_fs.fit(X_train, train["label"])
fs_preds = clf_fs.predict(X_test)
print("Fully supervised performance:")
# print(classification_report(y_test, fs_preds))

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(0)

In [16]:
y_test

0      0
1      0
2      0
3      0
4      0
      ..
495    0
496    0
497    0
498    0
499    0
Name: label, Length: 500, dtype: int64

In [17]:
fs_preds

NameError: name 'fs_preds' is not defined